Notes:
there are sites that stores the records for humidity, precipitation, wind speed, temperature of a specific city. If we want to add those features to increase reliability and validity of our model, we should be able to do it manually or maybe someone can figure out a way to properly data scrape / data mine. We can even add a boolean for whether it rained during that timeframe but that is more precise.

*   https://www.timeanddate.com/weather/philippines/makati/historic
*   https://www.wunderground.com/history/daily/ph/makati/IMAKAT1/date/2022-6-5
*   https://weatherspark.com/m/136799/2/Average-Weather-in-February-in-Makati-City-Philippines
*   https://www.accuweather.com/en/ph/makati-city/264878/february-weather/264878

For the incoming paper, here's a good reference:
https://scholarworks.rit.edu/cgi/viewcontent.cgi?article=12537&context=theses
<br>
Relevant but not for the paper:
https://www.sciencedirect.com/science/article/abs/pii/S0379711218303941

Goal:

Model with at least 60, but ideally 80% accuracy.
Right now, the status quo is that ohe is not enough, at least for basic models. we could also initially test our original df na formatted na into categorical into XDG models that allows categorical data for input but I don't expect much.

#  Data Preparation and Cleaning

In [ ]:
!pip install ydata_profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.9/357.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 18.9 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=e84b6e099b29d1be4fb48a1496889cd767b571f79bdfa804f96c9c49d52a766e
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.13.1
    Uninstalling seaborn-0.13.1:
      Successfully uninstalled seaborn-0.13.1


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
#from ydata_profiling import ProfileReport #one liner eda, not natively installed in google colab so do !pip install ydata_profiling or just remove this.
from sklearn import linear_model #I2
from sklearn.preprocessing import OneHotEncoder

In [ ]:
#Unfortunately for the case of location and i/e/e/o, the data is too disorganized to form some logic without too much complexity or nested if' statements, so we resort to manually adjusting the values into a uniform and standardized format.
URL = 'https://raw.githubusercontent.com/Vz-3/dataset_storage/master/Formatted-Firesight-Dataset.csv'
df = pd.read_csv(URL)

df.head(10)

HTTPError: HTTP Error 404: Not Found

<h3> Fundamentals </hr>

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info() #does df.dtypes, df.column, df.isna().sum() inverse

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated()

In [ ]:
df.nunique()

<h3> Cleaning </h3>

In [ ]:
"""
#Ignore this (had to reformat the day again since it has multiple redundancies of format, as shown by the next line of code) [unless if you want this to be added in cleansing]
print(df['DOTW'].value_counts())
print('Old:',df['DOTW'].value_counts().sum())
dayFormat = ['su','m','tu','w','th', 'f', 'sa']

toAdjust = [
    (df['DOTW'].isin(dayFormat)),
    (df['DOTW'] == 't') | (df['DOTW'] == 'tues'),
    (df['DOTW'] == 'sat') | (df['DOTW'] == 's'),
    (df['DOTW'] == 'sun')
]

values = [df['DOTW'],'tu','sa', 'su']
df['DOTW'] = np.select(toAdjust, values, df['DOTW']) #params: list of conditions, values for each condition, default value
df['DOTW'].astype('category')
print('New:',df['DOTW'].value_counts().sum())

#for brevity and readability
renameTo = ['DOTW','DATETIME','BRGY','ENTITIES','FIRE_CAUSE/ORIGIN','ALARM_LEVEL','DEATHS','INJURIES','RESPONDERS','ESTIMATED_PHP_LOSS']

df.rename(columns={col: new_name for col, new_name in zip(df.columns, renameTo)}, inplace=True)
df.columns

df['HOUR'] = df['DATETIME'].apply(lambda text: text[12:14])
df['DATETIME'] = df['DATETIME'].apply(lambda text: text[0:6])
df[['DAY','MONTH']] = df['DATETIME'].apply(lambda text: pd.Series(text.split(" ")))
df.drop(['DATETIME'],axis=1, inplace=True)

#Some minor issues in HOUR unique count so we turn them into integers to address the issue by first checking with pd.to_numeric(df['HOUR'], errors='coerce').notnull().all() to see if all the contents of the column is a numeric
df['HOUR'] = df['HOUR'].astype(int)

df.head()

#Now if we want to go back to a single column but with the datatype of datetime for time-series data with the loss of minutes,  we can do this by:
#there are a lot of benefits for converting numerical dates into pandas datetime, but IDK if we can properly utilize it since we only have 1 year and time-series requires a lot of data, that's why historical data is a big part of big data.
df['DATETIME'] = pd.to_datetime(
    df['MONTH'] + ' ' + df['DAY'].astype(str) + ', 2023 ' + df['HOUR'].astype(str) + ':00:00'
)
#delete the other columns
df.drop(columns=['HOUR','DAY','MONTH'],inplace=True)
#rearrange again
df = df[renameTo]
df.head()

print(f"Initial: {df['FIRE_CAUSE/ORIGIN'].nunique()}")

df['FIRE_CAUSE'] = df['FIRE_CAUSE/ORIGIN'].apply(lambda x: re.sub(r'((.*(?<=caused...)\s*)|(\s(?=cause:).*))|(.*(?<=from).)','',x) if (("caused" in x or "cause:" in x) or "from" in x) else x)

print(f'New: {df.FIRE_CAUSE.nunique()}')
df['FIRE_CAUSE'].head()

df['FIRE_ORIGIN'] = df['FIRE_CAUSE/ORIGIN'].apply(lambda x: re.sub(r'((\s(?=caused...).*)|(\s(?=cause:).*))|(\s(?=from).*)','',x) if (("caused" in x or "cause:" in x) or "from" in x) else x)
print(f'New: {df.FIRE_ORIGIN.nunique()}')
df['FIRE_ORIGIN'].head()

df['ESTIMATED_PHP_LOSS'] = df['ESTIMATED_PHP_LOSS'].apply(lambda val: int(float(re.sub('(php|,)','',val))))
df['ESTIMATED_PHP_LOSS'].head()

#for this case since there are some missing values, let's address them by replacing them with Negative.
df[['DEATHS','INJURIES']] = df[['DEATHS','INJURIES']].fillna('negative')
df.isna().sum()

df[['DEATHS','INJURIES']] = df[['DEATHS','INJURIES']].applymap(lambda foo: False if foo == 'negative' else True)
df.head(10)

df.nunique()
df[['DEATHS','INJURIES']] = df[['DEATHS','INJURIES']].astype('boolean')

df.drop(['RESPONDERS'], axis=1)
"""

In [ ]:
#It's cleaner and easier if we ignore case by turning all values into lowercase as well as removing extra spaces.
df = df.applymap(lambda text: re.sub(' +',' ',text.lower()) if isinstance(text, str) else text)
df.head()

In [ ]:
df['ENTITIES'].value_counts()

In [ ]:
#Let's remove the special characters to properly reflect the uniqueness of Entities.
df['ENTITIES'] = df['ENTITIES'].apply(lambda x: re.sub('_|-','',x))
df['ENTITIES'].value_counts()

In [ ]:
df['FIRE_ORIGIN'].value_counts()

In [ ]:
df['FIRE_CAUSE'].value_counts()

In [ ]:
df['ALARM_LEVEL'].value_counts()

In [ ]:
#df['ALARM_LEVEL'] = df['ALARM_LEVEL'].fillna('missing')

In [ ]:
#Converting into proper data types.
df['DOTW'] = df['DOTW'].astype('category')
df['MONTH'] = df['MONTH'].astype('category')
df['DAY'] = df['DAY'].astype('int')
df['ENTITIES'] = df['ENTITIES'].astype('category')
df['BRGY'] = df['BRGY'].astype('category')
df['FIRE_CAUSE'] = df['FIRE_CAUSE'].astype('category')
df['FIRE_ORIGIN'] = df['FIRE_ORIGIN'].astype('category')

df['ALARM_LEVEL'] = df['ALARM_LEVEL'].astype('category')

In [ ]:
#Reordering our columns for readability
df = df[['DOTW','DAY','HOUR','MONTH','BRGY','ENTITIES','FIRE_CAUSE','FIRE_ORIGIN','ALARM_LEVEL','DEATHS','INJURIES','ESTIMATED_PHP_LOSS']]
#Uncomment this if you want to download the formatted dataset once it's cleaned.
#df.to_csv('Formatted-Firesight-Dataset.csv',index=False)
df

# Exploratory Data Analysis


In [ ]:
# profiler = ProfileReport(df, title="Firesight Profiling Report")
# profiler.to_notebook_iframe()

In [ ]:
#try ko lang for initial visualization - (thea)
#it's actually advised that for EDA, we use tools like PowerBI or even webpivottable to help us visualize and derive analysis and insights -v, also no need to reimport the packages, yung mga bago lang.

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
msno.matrix(df)

In [ ]:
df.replace(np.NaN,0)

In [ ]:
entities_counts = df['ENTITIES'].value_counts()
top_entities = entities_counts.head(5)
labels = top_entities.index
counts = top_entities.values
plt.figure(figsize=(8, 8))
plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=140)
plt.axis('equal')
plt.title("TOP 5 INVOLVED ESTABLISHMENTS IN FIRE CASES - MAKATI CITY 2022")
plt.show()

In [ ]:
causes_counts = df['FIRE_ORIGIN'].value_counts()
top_causes = causes_counts.head(5)
labels = top_causes.index
counts = top_causes.values
plt.figure(figsize=(8, 8))
plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=140)
plt.axis('equal')
plt.title("TOP 5 ORIGIN OF FIRE CASES - MAKATI CITY 2022")
plt.show()

# Setup (Preprocessing, Balancing, etc)

In [ ]:
tempDf = df.copy()

In [ ]:
#Drop deaths and injuries
tempDf.drop(['DOTW','DEATHS','INJURIES', 'FIRE_CAUSE'], axis=1,inplace=True)
tempDf.shape

In [ ]:
labelEncodeDf = tempDf.copy()

In [ ]:
#All label encode test
labelEncodeDf = labelEncodeDf.apply(lambda col: col.astype('category').cat.codes if col.dtype == 'category' else col)
labelEncodeDf.info()

from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

y = labelEncodeDf['ESTIMATED_PHP_LOSS']
X = labelEncodeDf.loc[:, labelEncodeDf.columns != 'ESTIMATED_PHP_LOSS']

#Baseline model

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

lasso = Lasso()
lr = LinearRegression()
ridge = Ridge(alpha=1.0)

lasso.fit(X_train, y_train)
print(f'MAE: {mean_absolute_error(y_test,lasso.predict(X_test))}')
print(f'MSE: {mean_squared_error(y_test,lasso.predict(X_test))}')
print('Score: ',lasso.score(X_test, y_test))

lr.fit(X_train, y_train)
print(f'MAE: {mean_absolute_error(y_test,lr.predict(X_test))}')
print(f'MSE: {mean_squared_error(y_test,lr.predict(X_test))}')
print('Score: ',lr.score(X_test, y_test))

ridge.fit(X_train, y_train)
print(f'MAE: {mean_absolute_error(y_test,ridge.predict(X_test))}')
print(f'MSE: {mean_squared_error(y_test,ridge.predict(X_test))}')
print('Score: ',ridge.score(X_test, y_test))

In [ ]:
# profiler = ProfileReport(labelEncodeDf,title="Firesight Profiling Report: LE")
# profiler.to_notebook_iframe()

In [ ]:
tempDf.info()

In [ ]:
tempDf.nunique()

BRGY

In [ ]:
districtTwo = ['cembo','comembo','east rembo','guadalupe nuevo','guadalupe viejo','pembo','pinagkaisahan','pitogo','southside palar','rizal','south cembo','west rembo']

In [ ]:
tempDf['DISTRICT1'] = tempDf['BRGY'].apply(lambda brgy: 0 if brgy in districtTwo else 1) #direct encoding (because it's boolean)

In [ ]:
tempDf.drop('BRGY',axis=1,inplace=True)
tempDf.head()

FIRE_ORIGIN

In [ ]:
frequentOrigin = ['electrical ignition', 'open flame', 'lighted cigarette', 'overheated engine', 'ignition of materials', 'battery short circuit']

In [ ]:
tempDf['FIRE_ORIGIN'] = tempDf['FIRE_ORIGIN'].apply(lambda fire_origin: fire_origin if fire_origin in frequentOrigin else 'others')

In [ ]:
tempDf['FIRE_ORIGIN'].value_counts()

In [ ]:
#Strategies
#MONTH - nominal data, likely just label encode
#BRGY - must be label encode or target encode (Otherwise generalize into district 1 and district 2 only)
#ENTITIES - can onehotencode or target
#Fire_Cause - delete
#Fire_Origin -

In [ ]:
encDf = tempDf.copy()
encDf.info()
tempDf.info()

In [ ]:
ohe = OneHotEncoder()
result = 0
tempDf['FIRE_ORIGIN'] = tempDf['FIRE_ORIGIN'].astype('category')
for col in tempDf.columns:
    if col == 'MONTH' or col == 'ALARM_LEVEL':
        continue
    else:
        print(col)
        if tempDf[col].dtypes == 'category':
            encData = ohe.fit_transform(tempDf[[col]]).toarray()
            newdf = pd.DataFrame(encData, columns=ohe.get_feature_names_out())
            encDf = pd.concat([encDf,newdf], axis=1)

In [ ]:
encDf = encDf.drop(['MONTH','FIRE_ORIGIN','ENTITIES'],axis=1)
encDf.head()

In [ ]:
#profiler = ProfileReport(encDf,title="Firesight Profiling Report: OHE")
#profiler.to_notebook_iframe()

In [ ]:
encDf.nunique()
encDf.info()
encDf['ESTIMATED_PHP_LOSS'].value_counts()
encDf.to_csv('dataset.csv',index=False)

In [ ]:
#Baseline?
ye = encDf['ESTIMATED_PHP_LOSS']
Xe = encDf.loc[:, encDf.columns != 'ESTIMATED_PHP_LOSS']

X_train, X_test, y_train, y_test = train_test_split(Xe, ye, test_size=0.20)

lasso = Lasso()
lr = LinearRegression()
ridge = Ridge(alpha=1.0)

lasso.fit(X_train, y_train)
print(f'MAE: {mean_absolute_error(y_test,lasso.predict(X_test))}')
print(f'MSE: {mean_squared_error(y_test,lasso.predict(X_test))}')
print('Score: ',lasso.score(X_test, y_test))

lr.fit(X_train, y_train)
print(f'MAE: {mean_absolute_error(y_test,lr.predict(X_test))}')
print(f'MSE: {mean_squared_error(y_test,lr.predict(X_test))}')
print('Score: ',lr.score(X_test, y_test))

ridge.fit(X_train, y_train)
print(f'MAE: {mean_absolute_error(y_test,ridge.predict(X_test))}')
print(f'MSE: {mean_squared_error(y_test,ridge.predict(X_test))}')
print('Score: ',ridge.score(X_test, y_test))

# Data Modeling

Regression:

In [ ]:
encDf.head()

In [ ]:
def z_score_standardization(encDf):
    X_mean = np.mean(encDf)
    X_std = np.std(encDf)
    X_standardized = (encDf - X_mean) / X_std
    return X_standardized

In [ ]:
# Applying Z-Score Standardization
encDf[['DAY','HOUR']] = z_score_standardization(encDf[['DAY','HOUR']])

In [ ]:
from sklearn.neural_network import MLPRegressor

y_nn = encDf[['ESTIMATED_PHP_LOSS']]
X_nn = encDf.drop('ESTIMATED_PHP_LOSS',axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_nn, y_nn, test_size=0.33, random_state=1)

nn = MLPRegressor(
    activation='relu',
    hidden_layer_sizes=(10, 100),
    alpha=0.001,
    random_state=20,
    early_stopping=False
)

nn.fit(X_train, y_train)


pred = nn.predict(X_test)
test_set_rsquared = nn.score(X_test, y_test)
test_set_rmse = np.sqrt(mean_squared_error(y_test, pred))

print('R_squared value: ', test_set_rsquared)
print('RMSE: ', test_set_rmse)

In [ ]:
#Data preprocessing
#X = X_standardized.drop("ESTIMATED_PHP_LOSS", axis=1)
#y = X_standardized["ESTIMATED_PHP_LOSS"]

X = encDf.drop('ESTIMATED_PHP_LOSS',axis=1)
y = encDf["ESTIMATED_PHP_LOSS"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Linear Regression

In [ ]:
# training model
model_linreg = LinearRegression().fit(X_train, y_train)

# predicting the test set results
linreg_pred = model_linreg.predict(X_test)

#calculating the mean squared error
mse = mean_squared_error(y_test, linreg_pred)

#calculating the root mean squared error
rmse = mse ** 0.5

#calculating the r2 score
r2score = r2_score(y_test, linreg_pred)

print('Mean: ', np.mean(encDf['ESTIMATED_PHP_LOSS']))
print('Median: ', np.median(encDf['ESTIMATED_PHP_LOSS']))
print('Mean Squared Error: ', mse)
print('Root Mean Squared Error:', rmse)
print('R2 Score:', r2score)

Polynomial Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Polynomial Regression
degree = 3
poly_features = PolynomialFeatures(degree=degree)
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)
poly_reg_model = LinearRegression()
poly_reg_model.fit(X_train_poly, y_train)
y_pred_poly = poly_reg_model.predict(X_test_poly)

# Evaluation
mse_poly = mean_squared_error(y_test, y_pred_poly)
r2_poly = r2_score(y_test, y_pred_poly)
print("Polynomial Regression (Degree {}) - MSE: {:.4f}, R2: {:.4f}".format(degree, mse_poly, r2_poly))


Decision Tree Regression

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Decision Tree Regression
tree_reg_model = DecisionTreeRegressor()
tree_reg_model.fit(X_train, y_train)
y_pred_tree = tree_reg_model.predict(X_test)

# Evaluation
mse_tree = mean_squared_error(y_test, y_pred_tree)
r2_tree = r2_score(y_test, y_pred_tree)
print("Decision Tree Regression - MSE: {:.4f}, R2: {:.4f}".format(mse_tree, r2_tree))


Gradient Boosting Regression

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Gradient Boosting Regression
gb_reg_model = GradientBoostingRegressor()
gb_reg_model.fit(X_train, y_train)
y_pred_gb = gb_reg_model.predict(X_test)

# Evaluation
mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)
print("Gradient Boosting Regression - MSE: {:.4f}, R2: {:.4f}".format(mse_gb, r2_gb))

Classification

In [ ]:
clfDf = encDf.copy()
clfDf = clfDf.drop(['DAY','HOUR'], axis=1)
clfDf['ESTIMATED_PHP_LOSS'] = clfDf['ESTIMATED_PHP_LOSS'].apply(lambda val: 'low' if val < 100000 else ('medium' if val < 1000000 else 'high'))
clfDf['ESTIMATED_PHP_LOSS'].head()

Dealing with imbalance using oversampling and undersampling

In [ ]:
clfDf['ESTIMATED_PHP_LOSS'].value_counts()
X = clfDf.drop('ESTIMATED_PHP_LOSS',axis=1)
y = clfDf['ESTIMATED_PHP_LOSS']

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

resample_pipeline = Pipeline([
    ('oversample', SMOTE()),  # Increase the number of samples in the minority class to 50% of the majority class
    ('undersample', RandomUnderSampler()),  # Reduce the number of samples in the majority class to 80% of the minority class
])

X_resampled, y_resampled = resample_pipeline.fit_resample(X, y)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.33, random_state=42)

knn = KNeighborsClassifier(n_neighbors=11)
knn.fit(X_train, y_train)

knnPred = knn.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
class_labels = ['high', 'low', 'medium']
print("CL Report: ",classification_report(y_test, knnPred, zero_division=1))
confusionMatrix = confusion_matrix(y_test, knnPred, labels=class_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(confusionMatrix, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=class_labels,
            yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
X = clfDf[['FIRE_ORIGIN_battery short circuit', 'FIRE_ORIGIN_electrical ignition',
       'FIRE_ORIGIN_ignition of materials', 'FIRE_ORIGIN_lighted cigarette',
       'FIRE_ORIGIN_open flame', 'FIRE_ORIGIN_others',
       'FIRE_ORIGIN_overheated engine']]
y = clfDf['ESTIMATED_PHP_LOSS']

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.25, random_state=23)

knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(X_train, y_train)

knnPred2 = knn.predict(X_test)
print("CL Report: ",classification_report(y_test, knnPred2, zero_division=1))
confusionMatrix = confusion_matrix(y_test, knnPred2, labels=class_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(confusionMatrix, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=class_labels,
            yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# CLASSIFICATION + SIMULATION

last updated: aug 1 12:05 AM - vash n teya

In [ ]:
clfDf['ESTIMATED_PHP_LOSS'].value_counts() #target

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# X is the predictor and y is the target variable
X = tempDf[['FIRE_ORIGIN']]
y = clfDf['ESTIMATED_PHP_LOSS']

# Perform label encoding on the categorical features
label_encoder_X = LabelEncoder()
X_encoded = X.apply(label_encoder_X.fit_transform)

label_encoder_y = LabelEncoder()
y_encoded = label_encoder_y.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)
# Create a logistic regression model
model = LogisticRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")

In [ ]:
# Sample test data for simulation (replace this with your actual test data)
test_data = tempDf['FIRE_ORIGIN']

# Convert the test data to DataFrame
test_df = pd.DataFrame(test_data)

# Perform label encoding on the categorical features in the test data
test_X_encoded = test_df.apply(lambda col: label_encoder_X.transform(col))

# Make predictions on the test data
test_y_pred = model.predict(test_X_encoded)

# Convert the predicted labels back to their original classes for visualization
predicted_classes = label_encoder_y.inverse_transform(test_y_pred)

# Add the predicted classes to the test DataFrame
test_df['Predicted_ESTIMATED_PHP_LOSS'] = predicted_classes

test_df.head(10)


In [ ]:
# Sample test data for simulation (replace this with your actual test data)
test_data = tempDf['FIRE_ORIGIN']

# Convert the test data to DataFrame
test_df = pd.DataFrame(test_data)

# Perform label encoding on the categorical features in the test data
test_X_encoded = test_df.apply(lambda col: label_encoder_X.transform(col))

# Make predictions on the test data
test_y_pred = model.predict(test_X_encoded)

# Convert the predicted labels back to their original classes for visualization
predicted_classes = label_encoder_y.inverse_transform(test_y_pred)

# Add the predicted classes to the test DataFrame
test_df['ESTIMATED_PHP_LOSS'] = predicted_classes

# Get the count of 'low', 'medium', and 'high' for each unique value of 'FIRE_ORIGIN'
result_df = test_df.groupby(['FIRE_ORIGIN', 'ESTIMATED_PHP_LOSS']).size().unstack(fill_value=0)

# Add missing columns for 'medium' and 'high' and fill with 0 if not present
result_df = result_df.reindex(columns=['low', 'medium', 'high'], fill_value=0)

# Display the result
print(result_df)


# Data Evaluation

# References: